In [2]:
from sqlite3.dbapi2 import connect
import pandas as pd
import sqlite3

#load csv file
orders=pd.read_csv("orders.csv")
#load json file
users=pd.read_json("users.json")

#connect to restaurants sql file
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()

#read sql script from given file
with open("restaurants.sql","r") as f:
    sql_script=f.read()
    cursor.executescript(sql_script)
conn.commit()



In [4]:
#checking if sql script is executed
check=pd.read_sql_query("SELECT * FROM restaurants LIMIT 5",conn)
print(check)

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [7]:
restaurants=pd.read_sql_query("SELECT * FROM restaurants",conn)

#performing joins to creatr final csv file
final_df=(
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

#biuld final csv from dataframe
final_df.to_csv("final_food_delivery_dataset.csv")



In [13]:
final_df.describe()

,order_id,user_id,restaurant_id,total_amount,rating
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1504.117700,251.016700,801.162412,4.045430
std,2886.89568,861.727776,144.622558,405.458753,0.606531
min,1.00000,1.000000,1.000000,100.200000,3.000000
25%,2500.75000,761.000000,127.000000,446.310000,3.500000
50%,5000.50000,1508.000000,251.000000,806.295000,4.100000
75%,7500.25000,2250.250000,376.000000,1149.227500,4.600000
max,10000.00000,3000.000000,500.000000,1499.830000,5.000000


In [14]:
#Which city has the highest total revenue (total_amount) from Gold members?

gold_members_revenue=(final_df[final_df["membership"]=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False))

gold_members_revenue.head(1)

,total_amount
city,
Chennai,1080909.79


In [16]:
#Which cuisine has the highest average order value across all orders?
cuisine_avg=(
    final_df
    .groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

top_cuisine = cuisine_avg.idxmax()
top_value = cuisine_avg.max()

print("highest average order value:", top_cuisine)
print("avg order value:", top_value)



highest average order value: Mexican
avg order value: 808.0213444401395


In [17]:
#How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

user_total = (
    final_df
    .groupby("user_id")["total_amount"]
    .sum()
)

#users with>1000
count_users=(user_total > 1000).sum()

print(count_users)


2544


In [19]:
#Which restaurant rating range generated the highest total revenue?

# Create rating bins
bins=[3.0, 3.5, 4.0, 4.5, 5.0]
labels=["3.0–3.5", "3.5–4.0", "4.0–4.5", "4.5–5.0"]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# Calculate total revenue rating range
rating_revenue=(
    final_df
    .groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue.head(1)


/tmp/ipython-input-3836388122.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("rating_range")["total_amount"]


,total_amount
rating_range,
4.5–5.0,2197030.75


In [21]:
#Among Gold members, which city has the highest average order value?

gold_city_orders=(
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["order_id"]
    .count()
    .sort_values(ascending=False)
)

top_city=gold_city_orders.idxmax()
top_orders=gold_city_orders.max()

print(top_city)




Chennai


In [22]:
#Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

#get cuisine stats to get restaurant count for cuisines and count
cuisine_stats=(
    final_df
    .groupby("cuisine")
    .agg(
        restaurant_count=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
)

#sorting
cuisine_stats_sorted=cuisine_stats.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
)

cuisine_stats_sorted.head(1)



,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65


In [24]:
#What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

total_orders=len(final_df)

gold_orders=len(
    final_df[final_df["membership"] == "Gold"]
)

percentage=round((gold_orders / total_orders) * 100)

print( percentage)


50


In [41]:
# Which restaurant has the highest average order less than 20 total orders?

result=(
    orders
    .groupby("restaurant_name")
    .agg(
        total_orders=("order_id", "count")
    )
)

#restaurants with less than <20 orders
filtered=result[result["total_orders"] < 20]

#restaurant with highest order count
answer=filtered.sort_values(
    by="total_orders",
    ascending=False
)

print(answer.head(20))


                                    total_orders
restaurant_name                                 
Classic Kitchen Family Restaurant             19
Darbar Cafe Punjabi                           19
Classic Delights Family Restaurant            19
Classic Foods Pure Veg                        19
Classic Tiffins Pure Veg                      19
Darbar Eatery North Indian                    19
Darbar Delights Family Restaurant             19
Annapurna Tiffins Punjabi                     19
Annapurna Foods North Indian                  19
Annapurna Foods Non-Veg                       19
Sai Delights South Indian                     19
Royal Restaurant Andhra                       19
Ruchi Curry House Pure Veg                    19
Royal Tiffins Chinese                         19
Sai Cafe Family Restaurant                    19
Sai Curry House South Indian                  19
Ruchi Foods Chinese                           19
Ruchi Cafe Chinese                            19
Hotel Tiffins Punjab

In [43]:
#Which combination contributes the highest revenue?

combo_revenue=(
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
    .sort_values(by="total_amount", ascending=False)
)

print(combo_revenue)




  membership  cuisine  total_amount
7    Regular  Mexican    1072943.30
6    Regular  Italian    1018424.75
3       Gold  Mexican    1012559.79
2       Gold  Italian    1005779.05
5    Regular   Indian     992100.27
1       Gold   Indian     979312.31
0       Gold  Chinese     977713.74
4    Regular  Chinese     952790.91
Top combination:
Membership: Regular
Cuisine: Mexican
Total Revenue: 1072943.3


In [45]:
#During which quarter of the year is the total revenue highest?

final_df['order_date']=pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')
final_df['quarter']=final_df['order_date'].dt.quarter

quarterly_revenue=final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)

highest_revenue_quarter=quarterly_revenue.idxmax()
highest_revenue_amount=quarterly_revenue.max()

print(highest_revenue_quarter)


3


In [47]:
#How many total orders were placed by users with Gold membership?

gold_orders_count=final_df[final_df["membership"] == "Gold"].shape[0]

print(gold_orders_count)


Total orders placed by Gold members: 4987


In [49]:
#What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

hyderabad_revenue=(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"]
    .sum()
)

print(round(hyderabad_revenue))


1889367


In [50]:
#How many distinct users placed at least one order?

distinct_users=final_df["user_id"].nunique()

print(distinct_users)


2883


In [51]:
#What is the average order value (rounded to 2 decimals) for Gold members?

avg_gold_order_value=(
    final_df[final_df["membership"] == "Gold"]["total_amount"]
    .mean()
)

print(round(avg_gold_order_value, 2))


797.15


In [52]:
#How many orders were placed for restaurants with rating ≥ 4.5?
high_rating_orders=final_df[final_df["rating"] >= 4.5].shape[0]

print(high_rating_orders)


3374


In [53]:
#How many orders were placed in the top revenue city among Gold members only?

#find revenue first
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)
#get max revenue
top_city = gold_city_revenue.idxmax()

#count
order_count = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_city)
].shape[0]

print(top_city)
print(order_count)

Chennai
1337


In [55]:
#The total number of rows in the final merged dataset is __________.

final_df.shape[0]

10000